In [ ]:
import scipp as sc
import numpy as np
import dataconfig # run make_config.py to create this

Some helpers which still need to be added to scipp, ignore this:

In [ ]:
def to_bin_centers(d, dim):
    edges = d.coords[dim].copy()
    del d.coords[dim]
    d.coords[dim] = 0.5 * (edges[dim, 1:] + edges[dim, :-1])

In [ ]:
def to_bin_edges(d, dim):
    centers = d.coords[dim].copy()
    del d.coords[dim]
    first = 1.5*centers[dim, 0] - 0.5*centers[dim, 1]
    last = 1.5*centers[dim, -1] - 0.5*centers[dim, -2]
    bulk = 0.5 * (centers[dim, 1:] + centers[dim, :-1])
    edges = sc.concatenate(first, bulk, dim)
    edges = sc.concatenate(edges, last, dim)
    d.coords[dim] = edges

In [ ]:
def map_to_bins(data, dim, edges):
    data = data.copy()
    to_bin_edges(data, dim)
    bin_width = data.coords[dim][dim,1:] - data.coords[dim][dim,:-1]
    bin_width.unit = sc.units.one
    data *= bin_width
    data = sc.rebin(data, dim, edges)
    bin_width = edges[dim,1:] - edges[dim,:-1]
    bin_width.unit = sc.units.one
    data /= bin_width
    return data

## Setup and load data

In [ ]:
path = dataconfig.data_root
direct_beam_file = 'DirectBeam_20feb_full_v3.dat'
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
sample_run_number = 49338
sample_transmission_run_number = 49339
background_run_number = 49334
background_transmission_run_number = 49335

def load_larmor(run_number):
    return sc.neutron.load(filename=f'{path}/LARMOR000{run_number}.nxs')

def load_rkh(filename):
    return sc.neutron.load(
           filename=filename,
           mantid_alg='LoadRKH',
           mantid_args={'FirstColumnValue':'Wavelength'})

In [ ]:
%%time
sample_trans = load_larmor(sample_transmission_run_number)
sample = load_larmor(sample_run_number)
background_trans = load_larmor(background_transmission_run_number)
background = load_larmor(background_run_number)

In [ ]:
%%time
dtype = sample.coords['position'].dtype
sample_pos_offset = sc.Variable(value=[0.0, 0.0, 0.30530], unit=sc.units.m, dtype=dtype)
bench_pos_offset = sc.Variable(value=[0.0, 0.001, 0.0], unit=sc.units.m, dtype=dtype)
for item in [sample, sample_trans, background, background_trans]:
    item.coords['sample-position'] += sample_pos_offset
    item.coords['position'] += bench_pos_offset

In [ ]:
%%time
wavelength_bins = sc.Variable(
    dims=['wavelength'],
    unit=sc.units.angstrom,
    values=np.geomspace(0.9, 13.5, num=110))

In [ ]:
def apply_masks(data):
    tof = data.coords['tof']
    data.masks['bins'] = sc.less(tof['tof',1:], 1500.0 * sc.units.us) | \
                         (sc.greater(tof['tof',:-1], 17500.0 * sc.units.us) & \
                          sc.less(tof['tof',1:], 19000.0 * sc.units.us))
    pos = sc.neutron.position(data)
    x = sc.geometry.x(pos)
    y = sc.geometry.y(pos)
    data.masks['beam-stop'] = sc.less(sc.sqrt(x*x+y*y), 0.045 * sc.units.m)
    data.masks['tube-ends'] = sc.greater(sc.abs(x), 0.36 * sc.units.m) # roughly all det IDs listed in original
    #MaskDetectorsInShape(Workspace=maskWs, ShapeXML=self.maskingPlaneXML) # irrelevant tiny wedge?

In [ ]:
def background_mean(data, dim, begin, end):
    coord = data.coords[dim]
    assert (coord.unit == begin.unit) and (coord.unit == end.unit)
    i = np.searchsorted(coord, begin.value)
    j = np.searchsorted(coord, end.value) + 1
    return data - sc.mean(data[dim, i:j], dim)

In [ ]:
def transmission_fraction(incident_beam, transmission):
    # Approximation based on equations in CalculateTransmission documentation
    # TODO proper implementation of mantid.CalculateTransmission
    return (transmission / transmission) * (incident_beam / incident_beam)
    #CalculateTransmission(SampleRunWorkspace=transWsTmp,
    #                      DirectRunWorkspace=transWsTmp,
    #                      OutputWorkspace=outWsName,
    #                      IncidentBeamMonitor=1,
    #                      TransmissionMonitor=4, RebinParams='0.9,-0.025,13.5',
    #                      FitMethod='Polynomial',
    #                      PolynomialOrder=3, OutputUnfittedData=True)

In [ ]:
def extract_monitor_background(data, begin, end):
    background = background_mean(data, 'tof', begin, end)
    del background.coords['sample-position'] # ensure unit conversion treats this a monitor
    background = sc.neutron.convert(background, 'tof', 'wavelength')
    background = sc.rebin(background, 'wavelength', wavelength_bins)
    return background

def setup_transmission(data):
    incident_beam = extract_monitor_background(data['spectrum', 0], 40000.0*sc.units.us, 99000.0*sc.units.us)
    transmission = extract_monitor_background(data['spectrum', 3], 88000.0*sc.units.us, 98000.0*sc.units.us)
    return transmission_fraction(incident_beam, transmission)

In [ ]:
def solid_angle(data):
    # TODO proper solid angle
    # [0.0117188,0.0075,0.0075] bounding box size
    pixel_size = 0.0075 * sc.units.m 
    pixel_length = 0.0117188 * sc.units.m
    L2 = sc.neutron.l2(data)
    return (pixel_size * pixel_length) / (L2 * L2)

In [ ]:
def load_direct_beam(filenames, layers=None):
    """Load one or multiple direct beam files, and map to given layers.
    """
    if isinstance(filenames, str):
        filenames = [filenames]
    dbs = [ load_rkh(filename=f'{path}/{name}') for name in filenames ]
    if layers is None and len(dbs) == 1:
        return dbs[0]
    if layers is None:
        raise RuntimeError("Got multiple files but no layer mapping")
    direct_beam = None
    for db in dbs:
        if direct_beam is None:
            direct_beam = db
        else:
            direct_beam = sc.concatenate(direct_beam, db, 'layer')
    direct_beam.coords['layer'] = sc.Variable(dims=['layer'], values=np.arange(len(dbs)))
    return sc.choose(layer, choices=direct_beam, dim='layer')

In [ ]:
def to_q(data, transmission, direct_beam):
    transmission = setup_transmission(transmission)
    data = data.copy()
    apply_masks(data)
    data = sc.neutron.convert(data, 'tof', 'wavelength', out=data)
    data = sc.rebin(data, 'wavelength', wavelength_bins)

    monitor = data.attrs['monitor1'].value
    monitor = background_mean(monitor, 'tof', 40000.0*sc.units.us, 99000.0*sc.units.us)
    monitor = sc.neutron.convert(monitor, 'tof', 'wavelength', out=monitor)
    monitor = sc.rebin(monitor, 'wavelength', wavelength_bins)

    # this factor seems to be a fudge factor. Explanation pending.
    data *= 100.0 / 176.71458676442586

    # Setup direct beam and normalise to monitor. I.e. adjust for efficiency of detector across the wavelengths.
    # This would work assuming that there is a least one wavelength point per bin
    #direct_beam = sc.groupby(direct_beam, 'wavelength', bins=monitor.coords['wavelength']).mean('wavelength')
    direct_beam = map_to_bins(direct_beam, 'wavelength', monitor.coords['wavelength'])
    direct_beam = monitor * transmission * direct_beam
    
    # Estimate qresolution function
    moderator = load_rkh(filename=f'{path}/{moderator_file}')
    to_bin_edges(moderator, 'wavelength')
    # TODO
    #qResWs = TOFSANSResolutionByPixel(InputWorkspace=dataWs,
    #                                  DeltaR=8,
    #                                  SampleApertureRadius=4.0824829046386295,
    #                                  SourceApertureRadius=14.433756729740645,
    #                                  SigmaModerator=modWs, CollimationLength=5,
    #                                  AccountForGravity=True,
    #                                  ExtraLength=2)

    # TODO QResolution
    d = sc.Dataset({'data':data, 'norm':solid_angle(data)*direct_beam})
    to_bin_centers(d, 'wavelength')
    return sc.neutron.convert(d, 'wavelength', 'Q', out=d) # TODO no gravity yet

In [ ]:
def reduce(data, q_bins):
    data = sc.histogram(data, q_bins)
    if 'layer' in data.coords:
        return sc.groupby(data, 'layer').sum('spectrum')
    else:
        return sc.sum(data, 'spectrum')

In [ ]:
def reduce_by_wavelength(data, q_bins, wavelength_bands):
    # Cut range into number of requested bands
    n_band = int(wavelength_bands)
    n_bin = len(wavelength_bins.values)-1
    bounds = np.arange(n_bin)[::n_bin//n_band]
    bounds[-1] = n_bin
    slices =  [slice(i, j) for i,j in zip(bounds[:-1],bounds[1:])]
    bands = None
    # Reduce by wavelength slice
    for s in slices:
        band = sc.histogram(data['Q', s], q_bins)
        #band = sc.sum(band, 'spectrum')
        band = sc.groupby(band, 'layer').sum('spectrum')
        bands = sc.concatenate(bands, band, 'wavelength') if bands is not None else band
    # Add coord for wavelength edges of bands
    bands.coords['wavelength'] = sc.Variable(
        dims=['wavelength'],
        unit=sc.units.angstrom,
        values=np.take(wavelength_bins.values, bounds))
    return bands

## Direct beam
Choose one of the following options (run only one of the two cells) to load the direct beam file:
### Option 1: Same for all pixels

In [ ]:
direct_beam = load_direct_beam(direct_beam_file)

### Option 2: Different direct beam for different groups of pixels
Using randomly assigned IDs for this, change to something appropriate.
Filenames should be provided as a list, here we use same for all.

In [ ]:
%%time
layer = sc.Variable(dims=['spectrum'],
                    values=np.random.randint(low=0,
                                             high=11,
                                             size=len(sample.coords['spectrum'].values)))
direct_beam = load_direct_beam([direct_beam_file]*11, layers=layer)

## Q1D

### Step 1: Convert to Q
Note that this is not `I(Q)` yet.
Sum over spectra and normalization happens later.

In [ ]:
%%time
sample_q = to_q(data=sample, transmission=sample_trans, direct_beam=direct_beam)
background_q = to_q(data=background, transmission=background_trans, direct_beam=direct_beam)

### Step 2: Reduce (sum spectra)
Sum spectra. This does take into account potential different layers (see `'layers'` coord).
Note that this is still not `I(Q)`, normalization happens later.
We have two options, reduce the full wavelength range, or reduce individual wavelength bands.

In [ ]:
q_bins = sc.Variable(
    dims=['Q'],
    unit=sc.units.one/sc.units.angstrom,
    values=np.geomspace(0.008, 0.6, num=55))

In [ ]:
%%time
sample_q_all = reduce(sample_q, q_bins)
background_q_all = reduce(background_q, q_bins)

In [ ]:
%%time
sample_q_lambda = reduce_by_wavelength(sample_q, q_bins, wavelength_bands=8)
background_q_lambda = reduce_by_wavelength(background_q, q_bins, wavelength_bands=8)

Summing the latter over `'wavelength'` should give an equivalent result to the full wavelength result, but the former is faster if bands are not required.

### Step 3
There are multiple options for this step, depending on whether we need to consider wavelength bands and layers (pixel groups) independently or now.

#### Option 1: Combine all layers and normalize
This gives just a single `I(Q)` and can be compared to the default output from Mantid's `Q1D`.

In [ ]:
if 'layer' in sample_q_all.dims:
    sample_q1d = sc.sum(sample_q_all, 'layer')
    background_q1d = sc.sum(background_q_all, 'layer')
else:
    sample_q1d = sample_q_all
    background_q1d = background_q_all
sample_q1d = sample_q1d['data']/sample_q1d['norm']
background_q1d = background_q1d['data']/background_q1d['norm']
reduced = sample_q1d - background_q1d

reduced.attrs['UserFile'] = sc.Variable(
    value='USER_Raspino_191E_BCSLarmor_24Feb2020_v1.txt')
reduced.attrs['Transmission'] = sc.Variable(
    value=f'{sample_transmission_run_number}_trans_sample_0.9_13.5_unfitted')
reduced.attrs['TransmissionCan'] = sc.Variable(
    value=f'{background_transmission_run_number}_trans_can_0.9_13.5_unfitted')

In [ ]:
reduced

In [ ]:
from scipp.plot import plot
values, stddev = np.loadtxt("mantid_reduced.txt")
q = np.loadtxt("mantid_reduced_q.txt")

mantid = sc.DataArray(data=sc.Variable(['Q'],
                                       values=values,
                                       variances=stddev*stddev),
                      coords={'Q': sc.Variable(['Q'], unit=sc.units.one/sc.units.angstrom, values=q)})
mantid = sc.rebin(mantid, 'Q', reduced.coords['Q'])

ds = sc.Dataset({'mantid': mantid, 'scipp': reduced})
plot(ds, logy=True)

#### Option 2: Normalize without combining layers or wavelengths

Note that there are multiple options here: We can sum over `'layer'` or sum over `'wavelength'`, or neither.
As usualy, summation would need to be done *before* normalization, see option 1.
Here we do not perform any sum and obtain `I(wavelength, layer, Q)`:

In [ ]:
sample_q = sample_q_lambda['data']/sample_q_lambda['norm']
background_q = background_q_lambda['data']/background_q_lambda['norm']
reduced = sample_q - background_q

In [ ]:
reduced

In [ ]:
from scipp.plot import plot
plot(reduced['layer',0:4], collapse='Q', logy=True)
plot(reduced, log=True, vmin=-2, vmax=np.log(3.0)) # TODO fix https://github.com/scipp/scipp/issues/1112